<a href="https://colab.research.google.com/github/zubayerhimel/test/blob/master/Implementing_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
import tensorflow as tf
import keras
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import to_categorical
from keras.callbacks import LearningRateScheduler
from matplotlib import pyplot
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [ ]:
# load a single file as a numpy array
def load_file(filepath):
  dataframe = read_csv(filepath)
  return dataframe.values

In [ ]:
# importing file from drive
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
def dataPreprocessing(filePath, repeats):
  data = load_file(filePath)
  X, y = data[:, :-1], data[:, -1]
 
  # split into train test sets
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

  # Modify Train X 
  trainData = list()
  trainData.append(X_train)
  X_train = dstack(trainData)

  # Modify Train y
  y_train = np.reshape(y_train, (-1, 1))
  y_train = y_train - 1
  y_train = to_categorical(y_train);

  # Modify Test X 
  testData = list()
  testData.append(X_test)
  X_test = dstack(testData)

  # Modify Test y
  y_test = np.reshape(y_test, (-1, 1))
  y_test = y_test - 1
  y_test = to_categorical(y_test);

  # repeat experiment
  scores = list()
  for r in range(repeats):
    score = vanilla_LSTM(X_train, y_train, X_test, y_test)
    # score = stack_LSTM(X_train, y_train, X_test, y_test)
    score = score * 100.0
    print('Repeat number => %d: %.3f' % (r+1, score))
    scores.append(score)
	# summarize results
  summarize_results(scores)
  mse(y_train, y_test)

In [ ]:
# summarize scores
def summarize_results(scores):
	print(scores)
	m, s = mean(scores), std(scores)
	print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

In [ ]:
class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.lr = []
        
    def on_epoch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.lr.append(step_decay(len(self.losses)))
        print('lr:', step_decay(len(self.losses)))


In [ ]:
def step_decay(epoch):
    initial_lrate = 0.004
    drop = 0.5
    epochs_drop = 10.0
    lrate = initial_lrate * math.pow(drop, math.floor((epoch)/epochs_drop))
    return lrate

In [ ]:
loss_history = LossHistory()
lrate = LearningRateScheduler(step_decay)
callbacks_list = [loss_history, lrate]

In [ ]:
# fit and evaluate a vanilla lstm model
def vanilla_LSTM(X_train, y_train, X_test, y_test):
  verbose, epochs, batch_size = 0, 100, 128
  n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train.shape[1]
  print(n_timesteps)
  model = Sequential()
  model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
  model.add(Dropout(0.5))
  model.add(Dense(30, activation='relu'))
  model.add(Dense(n_outputs, activation='sigmoid'))
  opt = keras.optimizers.Adam(lr=0.004, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
  model.compile(optimizer = opt, loss='categorical_crossentropy', metrics=['accuracy'])
  model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=verbose, callbacks=callbacks_list)
  _, accuracy = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=verbose)
  
  return accuracy

In [ ]:
# fit and evaluate stack lstm
def stack_LSTM(X_train, y_train, X_test, y_test):
  verbose, epochs, batch_size = 0, 100, 128
  n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train.shape[1]
  print(n_timesteps)
  print(n_features)
  model = Sequential()

  model.add(LSTM(200, activation='relu', return_sequences=True, input_shape=(n_timesteps,n_features)))
  model.add(LSTM(100, activation='relu', return_sequences=True))
  model.add(LSTM(50, activation='relu', return_sequences=True))
  model.add(LSTM(25, activation='relu'))
  model.add(Dense(20, activation='relu'))
  model.add(Dense(10, activation='relu'))
  model.add(Dense(1))
  model.add(Dropout(0.5))

  model.add(Dense(n_outputs, activation='sigmoid'))
  tf.keras.utils.plot_model(model,to_file='model_plot.png', show_shapes=True)
  opt = keras.optimizers.Adam(lr=0.004, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
  model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
  model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=verbose)
  _, accuracy = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=0)
  
  return accuracy

In [ ]:
datasetList = list()
# datasetList.append('/content/gdrive/My Drive/Colab-Notebooks/dataset/single_files/walking1.csv')
# datasetList.append('/content/gdrive/My Drive/Colab-Notebooks/dataset/single_files/upstairs1.csv')
# datasetList.append('/content/gdrive/My Drive/Colab-Notebooks/dataset/single_files/standing1.csv')
# datasetList.append('/content/gdrive/My Drive/Colab-Notebooks/dataset/single_files/sitting1.csv')
# datasetList.append('/content/gdrive/My Drive/Colab-Notebooks/dataset/single_files/jogging1.csv')
# datasetList.append('/content/gdrive/My Drive/Colab-Notebooks/dataset/single_files/downstairs1.csv')
# datasetList.append('/content/gdrive/My Drive/Colab-Notebooks/dataset/single_files/walking2.csv')
# datasetList.append('/content/gdrive/My Drive/Colab-Notebooks/dataset/single_files/upstairs2.csv')
# datasetList.append('/content/gdrive/My Drive/Colab-Notebooks/dataset/single_files/standing2.csv')
# datasetList.append('/content/gdrive/My Drive/Colab-Notebooks/dataset/single_files/sitting2.csv')
# datasetList.append('/content/gdrive/My Drive/Colab-Notebooks/dataset/single_files/jogging2.csv')
# datasetList.append('/content/gdrive/My Drive/Colab-Notebooks/dataset/single_files/downstairs2.csv')
# datasetList.append('/content/gdrive/My Drive/Colab-Notebooks/dataset/merged2.csv')
datasetList.append('/content/gdrive/My Drive/Colab-Notebooks/dataset/merged.csv')

In [ ]:
for i in datasetList:
  dataPreprocessing(i, 5)

12
lr: 0.004
lr: 0.004
lr: 0.004
lr: 0.004
lr: 0.004
lr: 0.004
lr: 0.004
lr: 0.004
lr: 0.004
lr: 0.002
lr: 0.002
lr: 0.002
lr: 0.002
lr: 0.002
lr: 0.002
lr: 0.002
lr: 0.002
lr: 0.002
lr: 0.002
lr: 0.001
lr: 0.001
lr: 0.001
lr: 0.001
lr: 0.001
lr: 0.001
lr: 0.001
lr: 0.001
lr: 0.001
lr: 0.001
lr: 0.0005
lr: 0.0005
lr: 0.0005
lr: 0.0005
lr: 0.0005
lr: 0.0005
lr: 0.0005
lr: 0.0005
lr: 0.0005
lr: 0.0005
lr: 0.00025
lr: 0.00025
lr: 0.00025
lr: 0.00025
lr: 0.00025
lr: 0.00025
lr: 0.00025
lr: 0.00025
lr: 0.00025
lr: 0.00025
lr: 0.000125
lr: 0.000125
lr: 0.000125
lr: 0.000125
lr: 0.000125
lr: 0.000125
lr: 0.000125
lr: 0.000125
lr: 0.000125
lr: 0.000125
lr: 6.25e-05
lr: 6.25e-05
lr: 6.25e-05
lr: 6.25e-05
lr: 6.25e-05
lr: 6.25e-05
lr: 6.25e-05
lr: 6.25e-05
lr: 6.25e-05
lr: 6.25e-05
lr: 3.125e-05
lr: 3.125e-05
lr: 3.125e-05
lr: 3.125e-05
lr: 3.125e-05
lr: 3.125e-05
lr: 3.125e-05
lr: 3.125e-05
lr: 3.125e-05
lr: 3.125e-05
lr: 1.5625e-05
lr: 1.5625e-05
lr: 1.5625e-05
lr: 1.5625e-05
lr: 1.5625e-05
lr

ValueError: ignored